In [3]:
# Debug script to identify UVW discrepancy issues
import numpy as np
from astropy.coordinates import EarthLocation
import astropy.units as u
from pyuvdata import UVData
import matplotlib.pyplot as plt

def debug_uvw_discrepancy(hdf5_file, dsa110_location):
    """
    Debug function to identify sources of UVW discrepancies
    """
    print("=== UVW Discrepancy Debug ===")
    
    # Load the HDF5 file
    uvd = UVData()
    uvd.read(hdf5_file, file_type='uvh5', run_check=False)
    
    print(f"Original UVData info:")
    print(f"  Telescope name: {uvd.telescope_name}")
    print(f"  N antennas: {uvd.Nants_telescope}")
    print(f"  N baselines: {uvd.Nbls}")
    print(f"  Antenna names: {uvd.antenna_names[:10]}...")
    
    # Check telescope location
    if hasattr(uvd, 'telescope_location') and uvd.telescope_location is not None:
        tel_xyz = uvd.telescope_location
        print(f"  Telescope XYZ: {tel_xyz}")
        
        # Convert to EarthLocation for comparison
        tel_earthloc = EarthLocation.from_geocentric(
            tel_xyz[0] * u.m, tel_xyz[1] * u.m, tel_xyz[2] * u.m
        )
        print(f"  Telescope Lat/Lon/Alt: {tel_earthloc.lat.deg:.6f}°, {tel_earthloc.lon.deg:.6f}°, {tel_earthloc.height.value:.1f}m")
        
        # Compare with expected DSA-110 location
        try:
            expected_itrs = dsa110_location.get_itrs()
            expected_xyz = [expected_itrs.x.value, expected_itrs.y.value, expected_itrs.z.value]
        except:
            expected_xyz = [dsa110_location.itrs.x.value, dsa110_location.itrs.y.value, dsa110_location.itrs.z.value]
        
        location_diff = np.sqrt(np.sum((tel_xyz - expected_xyz)**2))
        print(f"  Location difference from expected DSA-110: {location_diff:.3f} m")
        
        if location_diff > 100:
            print(f"  ⚠️  LARGE telescope location discrepancy!")
    
    # Check antenna positions
    print(f"\nAntenna position analysis:")
    ant_pos = uvd.antenna_positions
    print(f"  Antenna positions shape: {ant_pos.shape}")
    print(f"  Position ranges:")
    print(f"    X: {ant_pos[:, 0].min():.1f} to {ant_pos[:, 0].max():.1f} m")
    print(f"    Y: {ant_pos[:, 1].min():.1f} to {ant_pos[:, 1].max():.1f} m") 
    print(f"    Z: {ant_pos[:, 2].min():.1f} to {ant_pos[:, 2].max():.1f} m")
    
    max_baseline = np.sqrt(np.sum(ant_pos**2, axis=1)).max()
    print(f"  Max antenna distance from center: {max_baseline:.1f} m")
    
    # Check UVW ranges
    uvw_max = np.max(np.abs(uvd.uvw_array))
    print(f"  Max UVW coordinate: {uvw_max:.1f} m")
    
    # Check for consistency
    if uvw_max > 10 * max_baseline:
        print(f"  ⚠️  UVW coordinates seem too large compared to antenna positions!")
    
    # Try to identify the coordinate system issue
    print(f"\nCoordinate system diagnosis:")
    
    # Check if antenna positions look like they're in absolute ECEF coordinates vs relative
    if max_baseline > 1e6:  # > 1000 km suggests absolute coordinates
        print(f"  ❌ Antenna positions appear to be in absolute ECEF coordinates")
        print(f"      (should be relative to telescope center)")
        return "absolute_coordinates"
    elif max_baseline < 10:  # < 10 m suggests wrong units or single antenna
        print(f"  ❌ Antenna positions seem too small - possibly wrong units")
        return "wrong_units"
    else:
        print(f"  ✓ Antenna position scale looks reasonable for relative coordinates")
    
    # Check time/LST consistency
    if hasattr(uvd, 'lst_array') and uvd.lst_array is not None:
        lst_range = uvd.lst_array.max() - uvd.lst_array.min()
        print(f"  LST range: {np.rad2deg(lst_range):.2f} degrees ({np.rad2deg(lst_range)/15:.2f} hours)")
        if lst_range > np.pi:  # > 12 hours
            print(f"  ⚠️  Very large LST range - might indicate time handling issues")
    
    return "needs_further_investigation"

def fix_telescope_location_reference(uvd, correct_location):
    """
    Fix telescope location reference issues
    """
    print("Fixing telescope location reference...")
    
    # Ensure telescope location is set correctly
    try:
        itrs_coord = correct_location.get_itrs() 
        correct_xyz = [itrs_coord.x.value, itrs_coord.y.value, itrs_coord.z.value]
    except:
        itrs_coord = correct_location.transform_to('itrs')
        correct_xyz = [itrs_coord.x.value, itrs_coord.y.value, itrs_coord.z.value]
    
    uvd.telescope_location = np.array(correct_xyz)
    
    # Also set the telescope_location_lat_lon_alt if available
    if hasattr(uvd, 'telescope_location_lat_lon_alt'):
        uvd.telescope_location_lat_lon_alt = (
            correct_location.lat.rad, 
            correct_location.lon.rad, 
            correct_location.height.to_value(u.m)
        )
    
    print(f"Set telescope location to: {correct_xyz}")
    return uvd

def check_antenna_subset_consistency(uvd, valid_antenna_names):
    """
    Check if antenna subsetting is causing issues
    """
    print("Checking antenna subset consistency...")
    
    # Check if all selected antennas have positions
    n_ants_data = uvd.Nants_data
    n_ants_telescope = uvd.Nants_telescope
    
    print(f"  Antennas in data: {n_ants_data}")
    print(f"  Antennas in telescope: {n_ants_telescope}")
    
    if n_ants_data != n_ants_telescope:
        print(f"  ⚠️  Mismatch between data antennas and telescope antennas")
        
        # Check if antenna_positions array matches data antennas
        if hasattr(uvd, 'antenna_positions') and uvd.antenna_positions.shape[0] != n_ants_data:
            print(f"  ❌ Antenna positions array size ({uvd.antenna_positions.shape[0]}) doesn't match data antennas ({n_ants_data})")
            return False
    
    return True

# Usage example:
def debug_your_hdf5(hdf5_file_path, dsa110_location):
    """
    Main debugging function for your specific case
    """
    result = debug_uvw_discrepancy(hdf5_file_path, dsa110_location)
    
    if result == "absolute_coordinates":
        print("\n🔧 SOLUTION: Convert antenna positions to relative coordinates")
        print("   Add this to your _load_uvh5_file function:")
        print("   # Convert absolute to relative antenna positions")
        print("   if np.max(np.abs(uvdata_obj.antenna_positions)) > 1e6:")
        print("       # Positions are in absolute ECEF, convert to relative")
        print("       tel_xyz = uvdata_obj.telescope_location")
        print("       uvdata_obj.antenna_positions -= tel_xyz")
        
    elif result == "wrong_units":
        print("\n🔧 SOLUTION: Check and fix antenna position units")
        
    print(f"\n💡 Additional recommendations:")
    print(f"   1. Verify telescope location is exactly correct")
    print(f"   2. Ensure antenna positions are relative to telescope center")
    print(f"   3. Check that LST calculation uses correct longitude")
    print(f"   4. Verify antenna selection doesn't break array consistency")
    
    return result

In [4]:
# Add this to your test script
from pathlib import Path

# Pick one HDF5 file to debug
hdf5_file = Path("/data/incoming/").glob("20*_sb00.hdf5").__next__()
result = debug_your_hdf5(str(hdf5_file), dsa110_utils.loc_dsa110)

=== UVW Discrepancy Debug ===


Telescope OVRO_MMA is not in known_telescopes.


Original UVData info:
  Telescope name: OVRO_MMA
  N antennas: 117
  N baselines: 4656
  Antenna names: ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']...
  Telescope XYZ: [-2409555.9334086  -4478213.61528579  3838767.45954561]
  Telescope Lat/Lon/Alt: 37.233405°, -118.283000°, 1188.1m
  Location difference from expected DSA-110: 0.000 m

Antenna position analysis:
  Antenna positions shape: (117, 3)
  Position ranges:
    X: -1125.1 to 1018.5 m
    Y: -269.3 to 1521.3 m
    Z: -304.8 to 1465.5 m
  Max antenna distance from center: 2161.5 m
  Max UVW coordinate: 1828.1 m

Coordinate system diagnosis:
  ✓ Antenna position scale looks reasonable for relative coordinates
  LST range: 1.24 degrees (0.08 hours)

💡 Additional recommendations:
   1. Verify telescope location is exactly correct
   2. Ensure antenna positions are relative to telescope center
   3. Check that LST calculation uses correct longitude
   4. Verify antenna selection doesn't break array consistency


NameError: name 'logger' is not defined